In [2]:
import numpy as np
import polars as pl

In [3]:
%%time
df = pl.read_csv("/Users/sonle/Documents/Data/delivery_orders_march.csv")


CPU times: user 4.39 s, sys: 1.91 s, total: 6.3 s
Wall time: 1.34 s


In [4]:
WORKDAYS = '1111110'
HOLIDAYS = ['2020-03-08','2020-03-25', '2020-03-30', '2020-03-31']

GMT8_OFFSET = 3600 * 8
DURATION_1DAY = 3600 * 24

sla_matrix_1st_attempt = [
	[3, 5, 7, 7],
	[5, 5, 7, 7],
	[7, 7, 7, 7],
	[7, 7, 7, 7]
]

sla_matrix_2nd_attempt =[
	[3, 3, 3, 3],
	[3, 3, 3, 3],
	[3, 3, 3, 3],
	[3, 3, 3, 3]
]
locations = ["Metro Manila", "Luzon", "Visayas", "Mindanao"]
locations = [loc.lower() for loc in locations]
location_to_index = {loc: i for i, loc in enumerate(locations)}
print(location_to_index)

min_length = min(map(len, locations))
trunc_location_to_index = {loc[-min_length:]: i for i, loc in enumerate(locations)}
print(trunc_location_to_index)
map_to_dict = dict(enumerate(np.array(sla_matrix_1st_attempt).flatten()))

{'metro manila': 0, 'luzon': 1, 'visayas': 2, 'mindanao': 3}
{'anila': 0, 'luzon': 1, 'sayas': 2, 'danao': 3}


In [6]:
print(df)

shape: (3176313, 6)
┌────────────────┬────────────┬─────────────────┬────────────────┬────────────────┬────────────────┐
│ orderid        ┆ pick       ┆ 1st_deliver_att ┆ 2nd_deliver_at ┆ buyeraddress   ┆ selleraddress  │
│ ---            ┆ ---        ┆ empt            ┆ tempt          ┆ ---            ┆ ---            │
│ i64            ┆ i64        ┆ ---             ┆ ---            ┆ str            ┆ str            │
│                ┆            ┆ f64             ┆ f64            ┆                ┆                │
╞════════════════╪════════════╪═════════════════╪════════════════╪════════════════╪════════════════╡
│ 2215676524     ┆ 1583138397 ┆ 1.5834e9        ┆ null           ┆ Baging ldl BUE ┆ Pantranco      │
│                ┆            ┆                 ┆                ┆ NAVISTA,PATAG. ┆ vill. 417      │
│                ┆            ┆                 ┆                ┆ CAGA...        ┆ Warehouse#     │
│                ┆            ┆                 ┆                ┆     

In [32]:
%%time
(
	df.with_columns([
		pl.when(pl.col('buyeraddress').str.slice(-min_length).str.to_lowercase() == "anila")
		.then(pl.lit(0))
		.when(pl.col('buyeraddress').str.slice(-min_length).str.to_lowercase() == "luzon")
		.then(pl.lit(1))
		.when(pl.col('buyeraddress').str.slice(-min_length).str.to_lowercase() == "sayas")
		.then(pl.lit(2))
		.otherwise(pl.lit(3))
		.alias('buyer_index')
	])
	.drop('buyeraddress')
)

CPU times: user 2.13 s, sys: 17.7 ms, total: 2.15 s
Wall time: 2.17 s


orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,selleraddress,buyer_index
i64,i64,f64,f64,str,i32
2215676524,1583138397,1.5834e9,null,"""Pantranco vill...",0
2219624609,1583309968,1.5835e9,1.5838e9,"""BLDG 210A Mora...",0
2220979489,1583306434,1.5835e9,null,"""#66 150-C, DRI...",0
2221066352,1583419016,1.5836e9,null,"""999maII 201,26...",3
2222478803,1583318305,1.5835e9,null,"""G66MANILA Hiya...",1
2222597288,1583328320,1.5836e9,null,"""33 sta.mesa 19...",0
2222738456,1583148712,1.5834e9,1.5837e9,"""salcedo Midlan...",0
2224695304,1583127841,1.5838e9,null,"""Polar 75 mulaw...",0
2224704587,1583307779,1.5834e9,1.5837e9,"""skies dept. Cr...",1


In [19]:
df.with_columns([pl.col('buyeraddress').str.slice(-min_length).]
				 )

orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
i64,i64,f64,f64,str,str
2215676524,1583138397,1.5834e9,null,"""anila""","""Pantranco vill..."
2219624609,1583309968,1.5835e9,1.5838e9,"""anila""","""BLDG 210A Mora..."
2220979489,1583306434,1.5835e9,null,"""anila""","""#66 150-C, DRI..."
2221066352,1583419016,1.5836e9,null,"""""","""999maII 201,26..."
2222478803,1583318305,1.5835e9,null,"""luzon""","""G66MANILA Hiya..."
2222597288,1583328320,1.5836e9,null,"""anila""","""33 sta.mesa 19..."
2222738456,1583148712,1.5834e9,1.5837e9,"""anila""","""salcedo Midlan..."
2224695304,1583127841,1.5838e9,null,"""anila""","""Polar 75 mulaw..."
2224704587,1583307779,1.5834e9,1.5837e9,"""luzon""","""skies dept. Cr..."


In [ ]:
(df.select(
        [
            pl.col('buyeraddress').map(lambda x: )
        ]
))